### Imports and setup

In [ ]:
from asf_core_data import get_mcs_installations, load_preprocessed_epc_data

import pandas as pd
import numpy as np

In [ ]:
# change path to your own local version of EPC data
epc_path = "/Users/chris.williamson/Documents/ASF_data"

In [ ]:
mcs = get_mcs_installations("full")

In [ ]:
# convert date columns to datetime type
mcs["commission_date"] = pd.to_datetime(mcs["commission_date"])
mcs["INSPECTION_DATE"] = pd.to_datetime(mcs["INSPECTION_DATE"])

In [ ]:
# merge installation type columns and filter to domestic installations
mcs["installation_type"] = mcs["installation_type"].fillna(
    mcs["end_user_installation_type"]
)
mcs = mcs.loc[mcs.installation_type == "Domestic"].reset_index(
    drop=True
)

### What proportion of domestic records in the MCS database relate to new-build installations?

In [ ]:
# UPRNs of properties that have an EPC labelled as "new dwelling" (not necessarily the first one - see below)
new_uprns = [uprn for uprn in mcs.loc[mcs["TRANSACTION_TYPE"] == "new dwelling"]["UPRN"] if uprn != "unknown"]

In [ ]:
# filter to first records
# records not linked to an EPC are kept
first_records = (
    mcs
    .sort_values("INSPECTION_DATE")
    .groupby("original_mcs_index")
    .head(1)
    .sort_values("original_mcs_index")
)

In [ ]:
# find number of days between first recorded EPC inspection and HP commission
first_records["diff_epc_to_mcs"] = (
    first_records["commission_date"] - first_records["INSPECTION_DATE"]
).dt.days

# assume dwelling was built with HP if:
# - it has an EPC indicating that it is a new dwelling
# - time difference between first EPC inspection and HP installation is less than 1 year
first_records["assumed_hp_when_built"] = (
    first_records["UPRN"].isin(new_uprns)
) & (first_records["diff_epc_to_mcs"] < 365)

Proportion of domestic MCS installations that relate to new builds:

In [ ]:
first_records.assumed_hp_when_built.value_counts(normalize=True)

Raw numbers:

In [ ]:
first_records.assumed_hp_when_built.value_counts()

Top 5 installers of domestic new build installations:

In [ ]:
first_records.loc[first_records["assumed_hp_when_built"]]["installer_name"].value_counts().head()

Difference in average costs for domestic retrofits and new builds:

In [ ]:
first_records.groupby("assumed_hp_when_built").cost.mean()

### What proportion of properties in the EPC database that were built with a HP appear in the MCS database?

In [ ]:
epc = load_preprocessed_epc_data(epc_path, version="preprocessed", usecols=["UPRN", "TRANSACTION_TYPE", "INSPECTION_DATE", "HP_INSTALLED"])

In [ ]:
# filter to records of new builds with a heat pump
new_hp = epc.loc[(epc["TRANSACTION_TYPE"] == "new dwelling") & (epc["HP_INSTALLED"])]

In [ ]:
# replace missing or unknown UPRNs with different codes to avoid them appearing the same in both datasets
new_hp["UPRN"] = new_hp["UPRN"].replace("unknown", "unknown_epc").fillna("unknown_epc").astype("str")
mcs["UPRN"] = mcs["UPRN"].replace("unknown", "unknown_mcs").fillna("unknown_mcs").astype("str")

In [ ]:
new_hp["in_mcs"] = new_hp["UPRN"].isin(mcs["UPRN"])

In [ ]:
new_hp = new_hp.drop_duplicates("UPRN")

Proportions of EPC new builds that are in the MCS database:

In [ ]:
new_hp["in_mcs"].value_counts(normalize=True)

Raw numbers:

In [ ]:
new_hp["in_mcs"].value_counts()

Weird that there are more properties identified here than by starting with the MCS dataset - how can this happen?

In [ ]:
assumed_hp_when_built_uprns = first_records.loc[first_records["assumed_hp_when_built"]]["UPRN"]

weird = new_hp.loc[(new_hp["in_mcs"]) & (~new_hp["UPRN"].isin(assumed_hp_when_built_uprns))]

In [ ]:
mcs.loc[mcs["UPRN"].isin(weird["UPRN"])][["UPRN", "commission_date", "TRANSACTION_TYPE", "INSPECTION_DATE"]].sort_values(["UPRN", "INSPECTION_DATE"]).head(20)

In [ ]:
epc.loc[epc["UPRN"].isin(weird["UPRN"])].sort_values(["UPRN", "INSPECTION_DATE"]).head(20)

These properties seem to be ones with an EPC certificate that says that the property is a "new dwelling" and has a HP and either:
* an earlier certificate with inspection date >1 year before the MCS commission date, or
* an MCS commission date that is >1 year after the EPC inspection date

The former could indicate errors in the EPC dataset (why would a new dwelling have a previous certificate?)

The latter could be properties getting their HP replaced or getting a second HP installed, or errors in the MCS dataset.

### Side note: How many "new dwelling" EPCs have an earlier certificate?

In [ ]:
epc["certificate_n"] = epc.sort_values("INSPECTION_DATE").groupby("UPRN").cumcount()

In [ ]:
epc.loc[(epc["UPRN"] != "unknown") & (epc["TRANSACTION_TYPE"] == "new dwelling")]["certificate_n"].value_counts()

Proportion of "new dwelling" EPCs that have an earlier certificate:

In [ ]:
(epc.loc[(epc["UPRN"] != "unknown") & (epc["TRANSACTION_TYPE"] == "new dwelling")]["certificate_n"] >= 1).value_counts(normalize=True)

One extreme example:

In [ ]:
epc.loc[epc["certificate_n"] == 273]

This one property has 300 "new dwelling" certificates:

In [ ]:
epc.loc[(epc["UPRN"] == "384131") & (epc["TRANSACTION_TYPE"] == "new dwelling")]

Transaction types of first records for properties with a "new dwelling" certificate and an earlier certificate:

In [ ]:
bad_uprns = epc.loc[(epc["UPRN"] != "unknown") & (epc["TRANSACTION_TYPE"] == "new dwelling") & (epc["certificate_n"] >= 1)]["UPRN"]

epc.loc[(epc["UPRN"].isin(bad_uprns)) & (epc["certificate_n"] == 0)]["TRANSACTION_TYPE"].value_counts(normalize=True)